In [59]:
import numpy as np
import re

def parse_equation(equation, variables):
    coeffs = {var: 0 for var in variables}
    terms = re.findall(r'[+-]?\s*\d*\s*[a-zA-Z]\d*', equation)
    
    for term in terms:
        var = re.findall(r'[a-zA-Z]\d*', term)[0]
        coeff = re.findall(r'[+-]?\s*\d+', term)
        
        if coeff:
            coeff = int(coeff[0].replace(" ", ""))
        else:
            coeff = 1 if term[0] not in '-+' else int(term[0] + '1')
        
        coeffs[var] = coeff
    
    constant = re.findall(r'(?<=\=)\s*[+-]?\d+', equation)
    constant = int(constant[0]) if constant else 0
    
    return [coeffs[var] for var in variables], constant

def extract_variables(equations):
    variables = set(re.findall(r'[a-zA-Z]\d*', ' '.join(equations)))
    return sorted(variables, key=lambda x: (re.sub(r'\d+', '', x), int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else 0))

def create_matrix_form(equations):
    variables = extract_variables(equations)
    
    A = []
    b = []
    
    for equation in equations:
        coeffs, constant = parse_equation(equation, variables)
        A.append(coeffs)
        b.append(constant)
    
    return np.array(A, dtype=float), np.array(b, dtype=float), variables

def print_matrix(M, step_msg):
    print(f"\n{step_msg}")
    num_rows, num_cols = M.shape
    for i in range(num_rows):
        row_str = " ".join(f"{M[i, j]:8.2f}" for j in range(num_cols))
        print(f"[ {row_str} ]")

def gaussian_elimination(A, b):
    n = len(b)
    M = np.hstack([A, b.reshape(-1, 1)])
    
    print("\nInitial Augmented Matrix:")
    print_matrix(M, "Initial Augmented Matrix")

    for i in range(n):
        # Partial pivoting
        max_row_index = np.argmax(abs(M[i:, i])) + i
        # Swap rows
        # Skip if the pivot is zero
        if M[max_row_index, i] == 0:
            continue
        # if the pivot is not in the current row, swap the rows
        if i != max_row_index:
            M[[i, max_row_index]] = M[[max_row_index, i]]
            print(f"\nRow Swap: R{i+1} <-> R{max_row_index+1}")
            print_matrix(M, "After Row Swap")
        
        # Normalize the pivot row
        M[i] = M[i] / M[i, i]
        print(f"\nNormalize Row {i+1}")
        print_matrix(M, "After Normalizing Pivot Row")
        
        # Eliminate rows below
        for j in range(i + 1, n):
            if M[j, i] != 0:
                factor = M[j, i] / M[i, i]
                M[j] = M[j] - factor * M[i]
                print(f"\nEliminate Row {j+1} using Row {i+1}")
                print_matrix(M, "After Row Operation")

    return M

def back_substitution(M):
    n = M.shape[0]
    x = np.zeros(n)

    for i in range(n - 1, -1, -1):
        x[i] = M[i, -1] - np.dot(M[i, i + 1:n], x[i + 1:])
    
    return x

def solve_linear_system(equations):
    A, b, variables = create_matrix_form(equations)

    # Perform Gaussian elimination
    M = gaussian_elimination(A, b)

    # Perform back substitution
    x = back_substitution(M)

    return {var: sol for var, sol in zip(variables, x)}

def main():
    equations = [
        "4y + 2z = 1",
        "2x + 3y + 5z = 0",
        "3x + y + z = 11"
    ]

    solution = solve_linear_system(equations)

    print("\nFinal Solution:")
    for var, sol in solution.items():
        print(f"{var} = {sol:.2f}")

if __name__ == "__main__":
    main()


Initial Augmented Matrix:

Initial Augmented Matrix
[     0.00     4.00     2.00     1.00 ]
[     2.00     3.00     5.00     0.00 ]
[     3.00     1.00     1.00    11.00 ]

Row Swap: R1 <-> R3

After Row Swap
[     3.00     1.00     1.00    11.00 ]
[     2.00     3.00     5.00     0.00 ]
[     0.00     4.00     2.00     1.00 ]

Normalize Row 1

After Normalizing Pivot Row
[     1.00     0.33     0.33     3.67 ]
[     2.00     3.00     5.00     0.00 ]
[     0.00     4.00     2.00     1.00 ]

Eliminate Row 2 using Row 1

After Row Operation
[     1.00     0.33     0.33     3.67 ]
[     0.00     2.33     4.33    -7.33 ]
[     0.00     4.00     2.00     1.00 ]

Row Swap: R2 <-> R3

After Row Swap
[     1.00     0.33     0.33     3.67 ]
[     0.00     4.00     2.00     1.00 ]
[     0.00     2.33     4.33    -7.33 ]

Normalize Row 2

After Normalizing Pivot Row
[     1.00     0.33     0.33     3.67 ]
[     0.00     1.00     0.50     0.25 ]
[     0.00     2.33     4.33    -7.33 ]

Eliminate 